# 1. 代价函数

## 1.1 二次代价函数
把label减去预测值的误差的平方累加起来再求平均值
$$C=\frac1{2n}\sum_x{\lVert{y(x)-a^L(x)}\rVert^2}$$
其中，C表示代价函数，x表示样本，y表示实际值，a表示输出值，n表示样本的总数。为简单起见，以一个样本为例进行说明，此时二次代价函数为：
$$C=\frac{(y-a)^2}2$$
* $a=\sigma(z), z=\sum{W_j*X_j}+b$
* $\sigma()$是激活函数

假如我们使用梯度下降法(Gradient descent)来调整权值参数的大小，权值w和偏置b的梯度推导如下：
$$\frac{\partial{C}}{\partial{w}}=(a-y)\sigma'(z)x$$

$$\frac{\partial{C}}{\partial{b}}=(a-y)\sigma'(z)$$

其中，z表示神经元的输入，σ表示激活函数。w和b的梯度跟激活函数的梯度成正比，激活函数的梯度越大，w和b的大小调整得越快，训练收敛得就越快。反之，如果激活函数的梯度几乎等于0，那么w和b几乎不会调整，这就是 **gradient vanish**。

## 1.2 交叉熵代价函数

假设我们的激活函数是sigmoid函数
![](http://upload-images.jianshu.io/upload_images/1791718-29de59f2ebc8571b.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
* 假设我们的目标是收敛到1，A点为0.82，离目标较B点更远，而A点处的梯度也比B点处大，权值调整比较大。B点为0.98，离目标较近，梯度比较小，权值调整也比较小，调整方案合理；
* 假设我们的目标是收敛到0，A点为0.82，离目标较B点更近，而A点处的梯度比B点处大，权值调整比较大。B点为0.98，离目标较远，梯度却比较小，权值调整也比较小，调整方案不合理；

换一个思路，我们不改变激活函数，而是改变代价函数，改用交叉熵代价函数：
$$C=-\frac1n\sum_x{[ylna+(1-y)ln(1-a)]}$$
其中，C表示代价函数，x表示样本，y表示实际值，a表示输出值，n表示样本的总数。
* $a=\sigma(z), z=\sum{W_j*X_j}+b$
* $\sigma'(z)=\sigma(z)(1-\sigma(z))$

![](http://upload-images.jianshu.io/upload_images/1791718-ee3d72911c4506d6.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

* 权值和偏置值的调整与$\sigma'(z)$无关，另外，梯度公式中的$\sigma(z)-y$表示输出值与实际值的误差。所以当误差越大时，梯度就越大，参数w和b的调整就越快，训练的速度也就越快。这样是比较合理的。
* 如果输出神经元是线性的，那么二次代价函数就是一种合适的选择。如果输出神经元是sigmoid函数，那么比较适合用交叉熵代价函数。

## 1.3 对数似然代价函数

对数似然函数常用来作为softmax回归的代价函数，如果输出层神经元是sigmoid函数，可以采用交叉熵代价函数。而深度学习中更普遍的做法是将softmax作为最后一层，此时常用的代价函数是对数似然函数。

对数似然代价函数与softmax的组合和交叉熵与sigmoid函数的组合非常相似。对数似然代价函数在二分类时可以化简为交叉熵代价函数的形式。

在TensorFlow中用：
* `tf.nn.sigmoid_cross_entropy_with_logits()`来表示跟sigmoid搭配使用的交叉熵。
* `tf.nn.softmax_cross_entropy_with_logits()`来表示跟softmax搭配使用的交叉熵。

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#创建一个简单的神经网络
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x,W)+b)

#二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))



# 2. 过拟合

防止过拟合
* 增加数据集
* 正则化
* Dropout

**Dropout**

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)

#创建一个简单的神经网络

# 2000个神经元的隐藏层
W1 = tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob) 

# 2000个神经元的隐藏层
W2 = tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob) 

# 1000个神经元的隐藏层
W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop = tf.nn.dropout(L3,keep_prob) 

# output
W4 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)

#二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(test_acc) +",Training Accuracy " + str(train_acc))


# 3. 优化器

## 3.1 Optimizer
```
tf.train.GradientDescentOptimizer

tf.train.AdadeltaOptimizer

tf.train.AdagradOptimizer

tf.train.AdagradDAOptimizer

tf.train.MomentumOptimizer

tf.train.AdamOptimizer

tf.train.FtrlOptimizer

tf.train.ProximalGradientDescentOptimizer

tf.train.ProximalAdagradOptimizer

tf.train.RMSPropOptimizer
```

## 3.2 各种优化器对比

**标准梯度下降法：**

标准梯度下降先计算所有样本汇总误差，然后根据总误差来更新权值

**随机梯度下降法：**

随机梯度下降随机抽取一个样本来计算误差，然后更新权值

**批量梯度下降法：**

批量梯度下降算是一种折中的方案，从总样本中选取一个批次（比如一共有10000个样本，随机选取100个样本作为一个batch），然后计算这个batch的总误差，根据总误差来更新权值。



In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#创建一个简单的神经网络
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x,W)+b)

#二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用Adam进行优化 学习率设置为0.01
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))
